In [1]:
#A* アルゴリズムによる8パズルの解法

'''
https://github.com/NiloofarShahbaz/8-puzzle-search-implementation
これを参考に一部改変した

また、今回の課題の目的はA*アルゴリズムの概要を理解することと考え、
A*アルゴリズムが適用されている関数を中心にコメントアウトをし、自身で解説した

'''

#ここから転載
from queue import PriorityQueue
from time import time
class Puzzle:
    goal_state=[1,2,3,8,0,4,7,6,5]#ゴールとする状態を定義
    heuristic=None
    evaluation_function=None
    needs_hueristic=False
    num_of_instances=0
    def __init__(self,state,parent,action,path_cost,needs_hueristic=False):
        self.parent=parent
        self.state=state
        self.action=action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost
        if needs_hueristic:
            self.needs_hueristic=True
            self.generate_heuristic()
            self.evaluation_function=self.heuristic+self.path_cost
        Puzzle.num_of_instances+=1

    def __str__(self):
        return str(self.state[0:3])+'\n'+str(self.state[3:6])+'\n'+str(self.state[6:9])

    def generate_heuristic(self): #ヒューリスティック関数の決定
        self.heuristic=0
        for num in range(1,9):
            distance=abs(self.state.index(num) - self.goal_state.index(num))#現在のコマの位置と正解の位置の距離差を算出
            #3×3の盤面をマンハッタン距離でコスト計算するためdistanceを3で割った商とあまりを算出
            i=int(distance/3)
            j=int(distance%3)
            self.heuristic=self.heuristic+i+j#元のヒューリスティック値にマンハッタン距離分のコストを追加して、ヒューリスティック値を更新
            

    def goal_test(self):
        if self.state == self.goal_state:
            return True
        return False

    @staticmethod
    def find_legal_actions(i,j):
        legal_action = ['U', 'D', 'L', 'R']
        if i == 0:  # up is disable
            legal_action.remove('U')
        elif i == 2:  # down is disable
            legal_action.remove('D')
        if j == 0:
            legal_action.remove('L')
        elif j == 2:
            legal_action.remove('R')
        return legal_action

    def generate_child(self):
        children=[]
        x = self.state.index(0)
        i = int(x / 3)
        j = int(x % 3)
        legal_actions=self.find_legal_actions(i,j)

        for action in legal_actions:
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x-3] = new_state[x-3], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x+3] = new_state[x+3], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x-1] = new_state[x-1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x+1] = new_state[x+1], new_state[x]
            children.append(Puzzle(new_state,self,action,1,self.needs_hueristic))
        return children

    def find_solution(self):
        solution = []
        solution.append(self.action)
        path = self
        while path.parent != None:
            path = path.parent
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution
    
    
def Astar_search(initial_state):
    count=0
    explored=[]#探検済みの状態を格納する関数
    start_node=Puzzle(initial_state,None,None,0,True)#nodeの生成
    q = PriorityQueue()#キュークラスのインスタンスを作成
    q.put((start_node.evaluation_function,count,start_node))
    #一つ目のキューに
    #初期状態の盤面の評価関数,count（世代情報）,盤面を入れる

    while not q.empty():#キューが空でないとき
        node=q.get()#キューから状態の情報を取り出す
        node=node[2]#取り出したキューの情報から盤面情報がある3番目を抽出
        explored.append(node.state)#探検済みに格納
        if node.goal_test():#そのノートがゴールでないかをチェック
            return node.find_solution()

        children=node.generate_child()#ノードから子を生成
        for child in children:#生成したノードが、探検済みでないときは世代情報を1加算して、キューに入れる
            if child.state not in explored:
                count += 1
                q.put((child.evaluation_function,count,child))
    return

#ここまで転載

state=[[1, 3, 4,
        8, 6, 2,
        7, 0, 5],

       [2, 8, 1,
        0, 4, 3,
        7, 6, 5],

       [2, 8, 1,
        4, 6, 3,
        0, 7, 5]]

for i in range(0,3):
    Puzzle.num_of_instances = 0
    t0 = time()
    astar = Astar_search(state[i])
    t1 = time() - t0
    print('A*:',astar)
    print('space:', Puzzle.num_of_instances)
    print('time:', t1)
    print()
    print('------------------------------------------')

A*: ['U', 'R', 'U', 'L', 'D']
space: 16
time: 0.0

------------------------------------------
A*: ['U', 'R', 'R', 'D', 'L', 'L', 'U', 'R', 'D']
space: 42
time: 0.0

------------------------------------------
A*: ['R', 'U', 'L', 'U', 'R', 'R', 'D', 'L', 'L', 'U', 'R', 'D']
space: 95
time: 0.0

------------------------------------------
